# Nextflow

[Nextflow](https://www.nextflow.io/) is a workflow management system used for executing scientific workflows across platforms scalably, portably, and reproducibly.

There are several options to automatically ingest Nextflow pipeline outputs into Lamindb instances:

1. Using a serverless environment trigger (e.g., AWS Lambda) to execute a Python script.
2. By using a [post-run script](https://docs.seqera.io/platform/23.4.0/launch/advanced#pre-and-post-run-scripts) on the Seqera Platform.

Both approaches execute a script that connects to the Lamindb instance and ingests the run output, similar to the one shown below.

This guide shows how to manually ingest a Nextflow run and illustrates what such an automated script could look like by registering the output of a run of the [mcmicro](https://github.com/labsyspharm/mcmicro) [Reference](https://mcmicro.org/) microscopy pipeline into a lamin instance.

In [ ]:
!lamin init --storage ./test-nextflow --name test-nextflow

In [ ]:
import lamindb as ln

## Run mcmicro pipeline

Run mcmicro pipeline and track input/output data with the script below:

In [ ]:
from IPython.display import display, Code

with open('mcmicro.py', 'r') as file:
    display(Code(file.read(), language='python'))

In [ ]:
!python mcmicro.py exemplar-001

## Data lineage

In [ ]:
output = ln.Artifact.filter(key__icontains="exemplar-001.ome.tif").one()

In [ ]:
output.view_lineage()

## View transforms and runs in LaminHub

[![hub](https://img.shields.io/badge/View%20in%20LaminHub-mediumseagreen)](https://lamin.ai/laminlabs/lamindata/transform/vMwsczN6lGZWRm8w/foyuuRRmEt7KYiaU8hPD)

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/FtEgTeQ9FXdbVWNnVTZ2.png" width="900px">

## View the database content

In [ ]:
ln.view()

In [ ]:
# clean up the test instance:
!rm -rf test-nextflow
!lamin delete --force test-nextflow